# Alter
- Linechart mit 3 Alterskohorten pro Bachelor/Master
- Stacked Barchart aggregiert über alle Studiengänge über die Jahre


In [55]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Dictionary zur Speicherung der Altersgruppen
alter_data = {"Jahr": []}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen

    # Studiengänge für Bachelor & Master automatisch erkennen
    bachelor_studiengaenge = df.filter(like="Bachelor").columns
    master_studiengaenge = df.filter(like="Master").columns

    # Gesamtanzahl der Studierenden pro Jahr für Gewichtung berechnen (nur für Variable "Alter")
    total_bachelor_students = df[
        (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
    ][bachelor_studiengaenge].sum(axis=1).values[0]

    total_master_students = df[
        (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
    ][master_studiengaenge].sum(axis=1).values[0]

    # Altersgruppen extrahieren
    altersgruppen = [
        "unter 18 Jahre (%)",
        "18 bis 20 Jahre (%)",
        "21 bis 23 Jahre (%)",
        "24 bis 26 Jahre (%)",
        "27 bis 29 Jahre (%)",
        "30 bis 32 Jahre (%)",
        "33 bis 35 Jahre (%)",
        "älter als 35 Jahre (%)"
    ]

    if jahr not in alter_data["Jahr"]:
        alter_data["Jahr"].append(jahr)

    for category in altersgruppen:
        row = df[
            (df["Variable"] == "Alter") & (df["Category"].str.contains(category, na=False, regex=False))
        ]
        if not row.empty:
            # **Absoluten Wert für jede Altersgruppe pro Studiengang berechnen**
            abs_bachelor = (row[bachelor_studiengaenge] / 100) * df[
                (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
            ][bachelor_studiengaenge].iloc[0]

            abs_master = (row[master_studiengaenge] / 100) * df[
                (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
            ][master_studiengaenge].iloc[0]

            # **Gesamte Anzahl an Studierenden in dieser Altersgruppe berechnen**
            total_abs_bachelor = abs_bachelor.sum().sum()
            total_abs_master = abs_master.sum().sum()

            # **Gewichteter relativer Anteil berechnen**
            weighted_bachelor = (total_abs_bachelor / total_bachelor_students) * 100
            weighted_master = (total_abs_master / total_master_students) * 100

            # Spaltennamen dynamisch hinzufügen
            bachelor_col = f"Bachelor {category}"
            master_col = f"Master {category}"

            if bachelor_col not in alter_data:
                alter_data[bachelor_col] = []
            if master_col not in alter_data:
                alter_data[master_col] = []

            alter_data[bachelor_col].append(weighted_bachelor)
            alter_data[master_col].append(weighted_master)

# Daten als DataFrame formatieren
df_alter = pd.DataFrame(alter_data).sort_values(by="Jahr")

In [17]:
df_alter

,Jahr,Bachelor unter 18 Jahre (%),Master unter 18 Jahre (%),Bachelor 18 bis 20 Jahre (%),Master 18 bis 20 Jahre (%),Bachelor 21 bis 23 Jahre (%),Master 21 bis 23 Jahre (%),Bachelor 24 bis 26 Jahre (%),Master 24 bis 26 Jahre (%),Bachelor 27 bis 29 Jahre (%),Master 27 bis 29 Jahre (%),Bachelor 30 bis 32 Jahre (%),Master 30 bis 32 Jahre (%),Bachelor 33 bis 35 Jahre (%),Master 33 bis 35 Jahre (%),Bachelor älter als 35 Jahre (%),Master älter als 35 Jahre (%)
0,2013,0.000000,0.0,21.674419,0.000000,55.331395,20.204878,14.938953,57.136585,3.511628,13.321951,2.325581,8.814634,0.569767,0.478049,1.040698,0.478049
1,2014,0.000000,0.0,2.357341,0.000000,41.952909,5.019608,39.975069,37.906863,9.930748,42.656863,2.617729,10.205882,2.318560,3.892157,0.806094,0.000000
2,2015,0.000000,0.0,28.120130,0.000000,48.522727,17.931624,15.097403,2666.786325,3.704545,14.341880,2.532468,3.487179,0.610390,0.000000,1.340909,1.820513
3,2016,1.916890,0.0,44.764075,1.762821,35.761394,28.724359,10.455764,42.262821,4.032172,17.871795,1.536193,7.121795,0.970509,1.910256,0.434316,0.000000
4,2017,0.332468,0.0,30.566234,0.000000,50.036364,16.384615,14.122078,52.471154,2.766234,20.735577,1.062338,6.312500,1.093506,2.298077,0.176623,1.966346
5,2018,2.452229,0.0,52.242038,0.000000,34.503185,34.747368,6.961783,27.473684,1.707006,10.747368,1.248408,3.263158,0.000000,1.052632,0.624204,0.000000
6,2019,0.000000,0.0,35.934783,0.000000,45.954106,19.641221,12.874396,50.774809,3.543478,20.507634,1.615942,5.896947,0.000000,1.122137,0.169082,0.732824
7,2020,0.000000,0.0,33.790191,0.000000,46.381471,18.688679,14.880109,47.179245,3.141689,17.377358,0.901907,8.386792,0.809264,2.391509,0.953678,3.773585
8,2021,0.000000,0.0,38.154229,0.468619,40.723881,21.154812,12.101990,38.414226,5.199005,21.133891,2.606965,10.866109,0.781095,3.811715,0.838308,3.012552
9,2022,1.963173,0.0,49.226629,0.956098,33.161473,45.317073,8.787535,31.229268,3.524079,14.126829,0.861190,2.326829,1.181303,2.892683,1.450425,2.853659


In [23]:
fig = px.line(
    df_alter, 
    x="Jahr", 
    y=[col for col in df_alter.columns if col != "Jahr"],  # Alle Altersgruppen außer "Jahr"
    markers=True, 
    title="Altersverteilung von Studierenden nach Kohorten (gewichteter Anteil)",
    labels={"value": "Anteil (%)", "variable": "Altersgruppe"},
    color_discrete_sequence=px.colors.qualitative.Safe  # Farbschema Safe verwenden
)

fig.update_layout(
    #template="infoviz",  # Dein eigenes Plotly-Template
    xaxis_title="Jahr",
    yaxis_title="Gewichteter Anteil der Studierenden (%)",
    yaxis=dict(range=[0, 100]),  # Prozentwerte begrenzen
    legend_title="Altersgruppe"
)

fig.show()

In [24]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Dictionary zur Speicherung der Altersgruppen
alter_data = {"Jahr": []}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen

    # Studiengänge für Bachelor & Master automatisch erkennen
    bachelor_studiengaenge = df.filter(like="Bachelor").columns
    master_studiengaenge = df.filter(like="Master").columns

    # Gesamtanzahl der Studierenden pro Jahr für Gewichtung berechnen (nur für Variable "Alter")
    total_bachelor_students = df[
        (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
    ][bachelor_studiengaenge].sum(axis=1).values[0]

    total_master_students = df[
        (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
    ][master_studiengaenge].sum(axis=1).values[0]

    # **Gesamtanzahl aller Studierenden berechnen**
    total_students = total_bachelor_students + total_master_students

    # Altersgruppen extrahieren
    altersgruppen = [
        "unter 18 Jahre (%)",
        "18 bis 20 Jahre (%)",
        "21 bis 23 Jahre (%)",
        "24 bis 26 Jahre (%)",
        "27 bis 29 Jahre (%)",
        "30 bis 32 Jahre (%)",
        "33 bis 35 Jahre (%)",
        "älter als 35 Jahre (%)"
    ]

    if jahr not in alter_data["Jahr"]:
        alter_data["Jahr"].append(jahr)

    for category in altersgruppen:
        row = df[
            (df["Variable"] == "Alter") & (df["Category"].str.contains(category, na=False, regex=False))
        ]
        if not row.empty:
            # **Absoluten Wert für jede Altersgruppe pro Studiengang berechnen**
            abs_bachelor = (row[bachelor_studiengaenge] / 100) * df[
                (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
            ][bachelor_studiengaenge].iloc[0]

            abs_master = (row[master_studiengaenge] / 100) * df[
                (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
            ][master_studiengaenge].iloc[0]

            # **Gesamte Anzahl an Studierenden in dieser Altersgruppe berechnen**
            total_abs_students = abs_bachelor.sum().sum() + abs_master.sum().sum()

            # **Gewichteter relativer Anteil berechnen (im Vergleich zur Gesamtzahl aller Studierenden)**
            weighted_total = (total_abs_students / total_students) * 100

            # Spaltennamen dynamisch hinzufügen
            col_name = f"{category}"  # Ohne Bachelor/Master Unterscheidung

            if col_name not in alter_data:
                alter_data[col_name] = []

            alter_data[col_name].append(weighted_total)

# Daten als DataFrame formatieren
df_alter = pd.DataFrame(alter_data).sort_values(by="Jahr")

In [31]:
fig = px.line(
    df_alter, 
    x="Jahr", 
    y=[col for col in df_alter.columns if col != "Jahr" and "Bachelor" in col], 
    markers=True, 
    title="Altersverteilung von Studierenden nach Kohorten (gewichteter Anteil)",
    labels={"value": "Anteil (%)", "variable": "Altersgruppe"},
    color_discrete_sequence=px.colors.qualitative.Safe  # Farbschema Safe verwenden
)

fig.update_layout(
    #template="infoviz",  # Dein eigenes Plotly-Template
    xaxis_title="Jahr",
    yaxis_title="Gewichteter Anteil der Studierenden (%)",
    yaxis=dict(range=[0, 50]),  # Prozentwerte begrenzen
    legend_title="Altersgruppe"
)

fig.show()

In [32]:
fig = px.line(
    df_alter, 
    x="Jahr", 
    y=[col for col in df_alter.columns if col != "Jahr" and "Master" in col], 
    markers=True, 
    title="Altersverteilung von Studierenden nach Kohorten (gewichteter Anteil)",
    labels={"value": "Anteil (%)", "variable": "Altersgruppe"},
    color_discrete_sequence=px.colors.qualitative.Safe  # Farbschema Safe verwenden
)

fig.update_layout(
    #template="infoviz",  # Dein eigenes Plotly-Template
    xaxis_title="Jahr",
    yaxis_title="Gewichteter Anteil der Studierenden (%)",
    yaxis=dict(range=[0, 50]),  # Prozentwerte begrenzen
    legend_title="Altersgruppe"
)

fig.show()

In [68]:
import pandas as pd
import plotly.express as px

# Daten für die Alterspyramide umformen
df_pyramide = df_alter.melt(id_vars="Jahr", var_name="Altersgruppe", value_name="Anteil (%)")

# Bachelor vs. Master identifizieren
df_pyramide["Studiengang"] = df_pyramide["Altersgruppe"].apply(lambda x: "Bachelor" if "Bachelor" in x else "Master")

# Altersgruppen umbenennen (Bachelor/Master entfernen)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Bachelor ", "", regex=False)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Master ", "", regex=False)

# Bachelor-Werte negativ setzen für visuelle Trennung
df_pyramide.loc[df_pyramide["Studiengang"] == "Bachelor", "Anteil (%)"] *= -1

# Neuen DataFrame erstellen mit nur den letzten Jahrgang
jahrgang = "2013"  # Letztes verfügbares Jahr
df_pyramide_latest = df_pyramide[df_pyramide["Jahr"] == jahrgang]

In [69]:
jahrgang

'2013'

In [70]:
fig = px.bar(
    df_pyramide_latest, 
    x="Anteil (%)", 
    y="Altersgruppe", 
    color="Studiengang", 
    orientation="h",  # Horizontal
    title=f"Alterspyramide der Studierenden ({jahrgang})",
    color_discrete_map={"Bachelor": "#1f77b4", "Master": "#ff7f0e"},  # Blautöne für Bachelor, Orange für Master
)

# Layout-Anpassungen
fig.update_layout(
    #template="infoviz",
    xaxis_title="Anteil der Studierenden (%)",
    yaxis_title="Altersgruppe",
    xaxis=dict(
        tickmode="array",
        tickvals=[-20, -10, 0, 10, 20],  # Achse zentrieren
        ticktext=["20%", "10%", "0%", "10%", "20%"]
    ),
    legend_title="Studiengang"
)

fig.show()

In [71]:
# Neuen DataFrame erstellen mit nur den letzten Jahrgang
jahrgang = "2024"  # Letztes verfügbares Jahr
df_pyramide_latest = df_pyramide[df_pyramide["Jahr"] == jahrgang]

In [72]:
fig = px.bar(
    df_pyramide_latest, 
    x="Anteil (%)", 
    y="Altersgruppe", 
    color="Studiengang", 
    orientation="h",  # Horizontal
    title=f"Alterspyramide der Studierenden ({jahrgang})",
    color_discrete_map={"Bachelor": "#1f77b4", "Master": "#ff7f0e"},  # Blautöne für Bachelor, Orange für Master
)

# Layout-Anpassungen
fig.update_layout(
    #template="infoviz",
    xaxis_title="Anteil der Studierenden (%)",
    yaxis_title="Altersgruppe",
    xaxis=dict(
        tickmode="array",
        tickvals=[-20, -10, 0, 10, 20],  # Achse zentrieren
        ticktext=["20%", "10%", "0%", "10%", "20%"]
    ),
    legend_title="Studiengang"
)

fig.show()

In [57]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Daten für die Alterspyramide umformen
df_pyramide = df_alter.melt(id_vars="Jahr", var_name="Altersgruppe", value_name="Anteil (%)")

# Bachelor vs. Master identifizieren
df_pyramide["Studiengang"] = df_pyramide["Altersgruppe"].apply(lambda x: "Bachelor" if "Bachelor" in x else "Master")

# Altersgruppen umbenennen (Bachelor/Master entfernen)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Bachelor ", "", regex=False)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Master ", "", regex=False)

# Bachelor-Werte negativ setzen für visuelle Trennung
df_pyramide.loc[df_pyramide["Studiengang"] == "Bachelor", "Anteil (%)"] *= -1

# Einzigartige Jahre für Subplots
jahre = sorted(df_pyramide["Jahr"].unique(), reverse=True)
num_years = len(jahre)

# Subplots erstellen
fig = make_subplots(
    rows=num_years, cols=1,  # Eine Spalte, eine Zeile pro Jahr
    shared_xaxes=True,  # Gleiche X-Achse für bessere Vergleichbarkeit
    subplot_titles=[f"Alterspyramide {jahr}" for jahr in jahre]
)

# Farben definieren
farben = {"Bachelor": "#1f77b4", "Master": "#ff7f0e"}  # Blau für Bachelor, Orange für Master

# Daten für jedes Jahr als separaten Subplot hinzufügen
for i, jahr in enumerate(jahre, start=1):
    df_year = df_pyramide[df_pyramide["Jahr"] == jahr]

    for studiengang in ["Bachelor", "Master"]:
        df_group = df_year[df_year["Studiengang"] == studiengang]

        fig.add_trace(
            go.Bar(
                x=df_group["Anteil (%)"],
                y=df_group["Altersgruppe"],
                orientation="h",
                name=studiengang if i == 1 else None,  # Legende nur einmal anzeigen
                marker=dict(color=farben[studiengang]),
            ),
            row=i, col=1
        )

# Layout anpassen
fig.update_layout(
    title="Alterspyramide der Studierenden über die Jahre",
    #template="infoviz",
    xaxis_title="Anteil der Studierenden (%)",
    height=300 * num_years,  # Höhe dynamisch anpassen
    showlegend=True,
)

# X-Achse so formatieren, dass negative und positive Werte gleichmäßig verteilt sind
fig.update_xaxes(
    tickmode="array",
    tickvals=[-20, -10, 0, 10, 20],
    ticktext=["20%", "10%", "0%", "10%", "20%"]
)

fig.show()

In [61]:
fig = px.bar(
    df_pyramide, 
    x="Jahr", 
    y="Anteil (%)", 
    color="Altersgruppe",
    title="Entwicklung der Altersverteilung über die Jahre",
    barmode="stack",  # Gestapelte Darstellung
    color_discrete_sequence=px.colors.qualitative.Safe
)

fig.update_layout(
    #template="infoviz",
    xaxis_title="Jahr",
    yaxis_title="Anteil der Studierenden (%)",
    yaxis=dict(range=[0, 10]),  # Alle Altersgruppen addieren sich zu 100%
    legend_title="Altersgruppe"
)

fig.show()